In [ ]:
import numpy as np
import cvxpy as cp
from scipy.linalg import sqrtm
import matplotlib.pyplot as plt
from AnsatzLacs import *
from CCAUtil import *

In [ ]:
sys_size = 2 # size of the input state (2 means 2 qubit)
ref_size = 0
xi_size = sys_size + 1

problem_seed = 1
swap_test_unitary = get_swap_unitary(sys_size)
swap_test_unitary_xi = get_swap_unitary(xi_size)

In [ ]:
# define rho and sigma
np.random.seed(problem_seed)

rho_ansatz = General(sys_size, ref_size, 2)
sigma_ansatz = General(sys_size, ref_size, 2)
rho_prob = QCBM(sys_size, ref_size, 2)
sigma_prob = QCBM(sys_size, ref_size, 2)

rho_param = np.random.normal(size=rho_ansatz.get_parameter_num()) % (2 * np.pi)
sigma_param = np.random.normal(size=sigma_ansatz.get_parameter_num()) % (2 * np.pi)
rho_prob_param = np.random.normal(size=rho_prob.get_parameter_num()) % (2 * np.pi)
sigma_prob_param = np.random.normal(size=sigma_prob.get_parameter_num()) % (2 * np.pi)

rho_ansatz.set_params(rho_param)
sigma_ansatz.set_params(sigma_param)
rho_prob.set_params(rho_prob_param)
sigma_prob.set_params(sigma_prob_param)

rho = CCA(rho_prob, rho_ansatz)
sigma = CCA(sigma_prob, sigma_ansatz)
rho_dm = rho.get_dm(None)
sigma_dm = sigma.get_dm(None)

In [ ]:
def get_root_fidelity(p, q):
    """
    returns \sqrt{F(p, q)}
    """
    sqrt_p = sqrtm(p)
    sqrt_q = sqrtm(q)
    X = sqrt_p @ sqrt_q
    X_dag = np.conjugate(X).T
    return np.real(np.sqrt(np.power(np.trace(sqrtm(X_dag @ X)), 2)))

In [ ]:
expected = get_root_fidelity(rho_dm, sigma_dm)
print("Exact root fidelity:", expected)

In [ ]:
# If want to check with cvxpy
n = 2**sys_size
Y = cp.Variable(rho_dm.shape, hermitian=True)
Z = cp.Variable(rho_dm.shape, hermitian=True)
V = cp.Variable((2*n, 2*n))
V = cp.bmat([[Y, np.eye(n)], [np.eye(n), Z]])

constraints = [Y >> 0, Z >> 0, V >> 0]
prob = cp.Problem(cp.Minimize(0.5*cp.real(cp.trace(Y @ rho_dm) + cp.trace(Z @ sigma_dm))),
                  constraints)

# Solve SDP
prob.solve(verbose=False)
# expected = prob.value
print("Aprroximated root fidelity:", prob.value)

In [ ]:
def get_trace(cca1, cca2, qcbm_shots, num_shots):
    return get_trace_helper(swap_test_unitary, cca1, cca2, qcbm_shots, num_shots)

In [ ]:
def get_gradient(params, fun):
    delta = 1 - 2 * np.random.binomial(1, 0.5, size=len(params))
    plus = fun(params + perturbation*delta)
    minus = fun(params - perturbation*delta)
    grad = (plus-minus)/(2*perturbation*delta)

    norm = np.linalg.norm(grad)
    if norm > 2:
        grad = grad/norm
    
    return grad

def gradient_descent(fun, x0, jac, maxiter, callback = None):
    x = x0
    for i in range(maxiter):
        iteration = i + 1
        
        x -= lr * jac(x, fun)
        
        if callback != None:
            callback(x, iteration)
    return x

def adam(fun, x0, jac, maxiter, beta1 = .9, beta2 = .9, callback = None):
    x = x0
    m = np.zeros(len(x))
    v = np.zeros(len(x))
    
    for i in range(maxiter):
        derivative = jac(x0, fun)
        
        m = beta1 * m + (1 - beta1) * derivative
        v = beta2 * v + (1 - beta2) * derivative * derivative
        lr_eff = lr * np.sqrt(1 - beta2**(i+1)) / (1 - beta1**(i+1))
        x -= lr_eff * m / np.sqrt(v)
    
        if callback != None:
            callback(x, i+1)
    return x

# Dual Optimization

In [ ]:
def get_trace_xi(x, qcbm_shots, shots):
    """
    Calculates term Tr[(|0><0|ω)ξ] and Tr[(|1><1|τ)ξ]
    If x = True, calculates Tr[(|1><1|τ)ξ].
    If x = False, calculates Tr[(|0><0|ω)ξ].
    """
    if x:
        app = np.array([[0, 0], [0, 1]])
        ansatz = tau
    else:
        app = np.array([[1, 0], [0, 0]])
        ansatz = omega
        
    if shots is None:
        xi_dm = xi.get_dm(None)
        a = ansatz.get_dm(None)
        return np.trace(np.real(np.kron(app, a)@xi_dm))
    
    else: 
        xi_dm = xi.get_dm(qcbm_shots)
        state2 = np.kron(app, ansatz.get_dm(qcbm_shots))

        total_state = np.kron(xi_dm,state2)
        density_matrix = swap_test_unitary_xi @ total_state @ np.conjugate(swap_test_unitary_xi).T

        probabilities = np.real(density_matrix.diagonal())
        return calculate_trace(probabilities, xi_size, shots)

In [ ]:
def get_trace_pauli(qcbm_shots, shots):
    '''
    Calculates the term Tr[(σ_x \otimes I)ξ]
    '''
    if shots is None:
        xi_dm = xi.get_dm(None)
        X = np.array([[0, 1], [1, 0]])
        I = np.eye(2 ** (xi_size - 1))
        return np.real(np.trace(np.kron(X, I) @ xi_dm))
    
    else:
        xi_dm = xi.get_dm(qcbm_shots)
        density_matrix = XI_unitary @ xi_dm @ np.conjugate(XI_unitary).T
        probabilities = np.real(density_matrix.diagonal())

        rng = np.random.default_rng()
        counts = rng.multinomial(shots, np.fromiter(probabilities, dtype=float))
        probabilities_shots = counts / shots

        return np.dot(probabilities_shots,get_pauli_val(pauli))

In [ ]:
N = 2 ** (sys_size + 1)

def get_dual_params(params):
    lmbda = params[-3]
    mu = params[-2]
    nu = params[-1]

    omega_params = params[:dual_num_parameters]
    tau_params = params[dual_num_parameters:2 * dual_num_parameters]
    xi_params = params[2 * dual_num_parameters: 2 * dual_num_parameters + dual_num_parameters_xi]
    
    omega.set_params(omega_params)
    tau.set_params(tau_params)
    xi.set_params(xi_params)
    return 

def dual_cost_function_full(params):
    get_dual_params(params)
    
    # calculates the objective function
    exp_omega_rho = get_trace(omega, rho, qcbm_shots, num_shots)
    exp_tau_sigma = get_trace(tau, sigma, qcbm_shots, num_shots)
    termA = lmbda * exp_omega_rho + mu * exp_tau_sigma
    
    exp_omega = get_purity(omega, qcbm_shots)
    exp_tau = get_purity(tau, qcbm_shots)
    exp_xi = get_purity(xi, qcbm_shots)
    termB = lmbda**2 * exp_omega + mu**2 * exp_tau + N + nu**2 * exp_xi
    
    exp_omega_xi = get_trace_xi(False, qcbm_shots, num_shots)
    exp_tau_xi = get_trace_xi(True, qcbm_shots, num_shots)
    exp_pauli_xi = get_trace_pauli(qcbm_shots, num_shots)
    termC = -2 * (lmbda * nu * exp_omega_xi + mu * nu * exp_tau_xi + nu * exp_pauli_xi)
    
    obj = termA / 2
    penal = termB + termC
    cost = obj + c * penal
    return cost, obj, penal

def dual_cost_function(params):
    """
    cost function for updating both ansatzs
    """
    cost, _, _ = dual_cost_function_full(params)
    return cost

In [ ]:
print_every = 100
def dual_callback(x, iteration):
    '''
    Callback function called automatically during optimization. 
    Appends loss value, penalties, and objective value.
    Print out the training status throughout the training.
    '''
    
    global lr
    global lr_wait
    
    global dual_loss
    global dual_penals
    global dual_ojectives
    global dual_iterations
    
    fx, obj, penal = dual_cost_function_full(x)
    dual_loss.append(fx)
    dual_penals.append(penal)
    dual_objectives.append(obj)
    dual_iterations.append(iteration)
        
    if (iteration % print_every == 0):
        lr_wait += 1
        if lr_wait >= 5:
            m, _ = np.polyfit(dual_iterations[-500:], dual_loss[-500:], 1)
            if m > 0:
                lr = max(lr/2,lr_min)
                lr_wait = 0
            else:
                lr = min(lr*1.1,lr_max)
        print(f"Iteration: {iteration}\tCost: {round(fx,4)}\tObj: {round(obj, 4)}\tPenalty: {round(penal, 4)}\tc: {round(c, 4)}\tlr: {round(lr, 4)}")


In [ ]:
# General settings
num_shots = None
qcbm_shots = None

max_iter = 200 * 1000
perturbation = 1e-4
c = 5

lr_max = 1e-1
lr_min = 1e-4

dual_num_layers = 4
dual_qcbm_layers = 3

dual_num_params_u = get_parameter_number("general", sys_size, ref_size, dual_num_layers)
dual_num_params_qcbm = get_parameter_number("qcbm", sys_size, ref_size, dual_qcbm_layers)
dual_num_parameters = dual_num_params_u + dual_num_params_qcbm

dual_num_params_u_xi = get_parameter_number("general", xi_size, ref_size, dual_num_layers)
dual_num_params_qcbm_xi = get_parameter_number("qcbm", xi_size, ref_size, dual_qcbm_layers)
dual_num_parameters_xi = dual_num_params_u_xi + dual_num_params_qcbm_xi

In [ ]:
num_train = 3
record = 0

iterations_dual_outer = []
costs_dual_outer = []
objs_dual_outer = []
penalties_dual_outer = []

for i in range(num_train):
    lr = lr_max
    lr_wait = 0
    
    lmbda = 1
    mu = 1
    nu = 1

    omega = create_cca("general", sys_size, ref_size, dual_num_layers, dual_qcbm_layers)
    tau = create_cca("general", sys_size, ref_size, dual_num_layers, dual_qcbm_layers)
    xi = create_cca("general", xi_size, ref_size, dual_num_layers, dual_qcbm_layers)
    
    print("Expected value:", expected)
    init_params = [*omega.get_params(), *tau.get_params(), *xi.get_params(), lmbda, mu, nu]
    init_cost, init_obj, init_penal = dual_cost_function_full(init_params)
    
    dual_loss = [init_cost]
    dual_penals = [init_penal]
    dual_objectives = [init_obj]
    dual_iterations = [0]
    
    print(f"Iteration {i}, Initial cost : {init_cost}")
    result = gradient_descent(dual_cost_function, init_params, get_gradient, max_iter, callback=dual_callback)
    
    costs_dual_outer.append(dual_loss)
    objs_dual_outer.append(dual_objectives)
    penalties_dual_outer.append(dual_penals)
    iterations_dual_outer.append(dual_iterations)
    
    
    if abs(dual_loss[-1] - expected) <= 0.05:
        dir_name = f"Fidelity_CCA_dual_data/seed{problem_seed}"
        problem_name = "Fid"
        filename = f"Init_{i}"
        save_data_to_file(dual_loss, dual_penals, dual_objectives, problem_name, dir_name, filename)
        record += 1
        
print("Number of successful trails: ", record)

# Primal Optimization

In [ ]:
omega_size = sys_size + 1
num_alpha = 16
swap_test_unitary_omega = get_swap_unitary(omega_size)

In [ ]:
rho_0 = np.kron(np.array([[1, 0], [0, 0]]), rho_dm)
sigma_1 = np.kron(np.array([[0, 0], [0, 1]]), sigma_dm)

def get_trace_omega(rho, qcbm_shots, shots):
    """
    Calculate terms Tr[(|0><0|ρ)ω] and Tr[(|1><1|τ)ω].
    If rho = True, calculates Tr[(|0><0|ρ)ω].
    If rho = False, calculates Tr[(|1><1|τ)ω].
    """
    state1 = rho_0 if rho else sigma_1
    
    # if noiseless simulation
    if shots is None:
        state_omega = omega.get_dm(None)
        return np.real(np.trace(state1 @ state_omega))
    
    # with shots
    else:
        state_omega = omega.get_dm(qcbm_shots)
        total_state = np.kron(state1, state_omega)
        density_matrix = swap_test_unitary_omega @ total_state @ np.conjugate(swap_test_unitary_omega).T

        probabilities = np.real(density_matrix.diagonal())
        return calculate_trace(probabilities, omega_size, shots)

In [ ]:
def int_to_pauli_string(x):
    return ("" if x < 4 else int_to_pauli_string(x//4)) + 'IXYZ'[x % 4]

x = list(range(4**sys_size))
pauli_strings = []
for i in range(4**sys_size):
    string = int_to_pauli_string(x[i])
    string = (sys_size-len(string))*'I' + string
    pauli_strings.append(string)

print(pauli_strings)

Xpaulis = ['X' + s for s in pauli_strings]
Ypaulis = ['Y' + s for s in pauli_strings]
all_paulis = Xpaulis + Ypaulis
expectation_unitary_dict = {pauli_string: get_expectation_unitary(pauli_string) for pauli_string in all_paulis}
prod_unitary_dict = {pauli_string: get_pauli_prod(pauli_string) for pauli_string in pauli_strings}

In [ ]:
def get_X(alpha):
    res = np.zeros((2**sys_size, 2**sys_size), dtype=np.cdouble)
    for i in range(len(alpha)):
        res += alpha[i] * prod_unitary_dict[pauli_strings[i]]
    return res

def get_pauli_omega_trace(alpha, qcbm_shots, shots):
    """
    calculate terms Tr[(|1><0|X)ω]"""
    if shots is None:
        X = get_X(alpha)
        omega_state = omega.get_dm(None)
        XY = np.array([[0, 0], [1, 0]], dtype=np.cdouble)
        return np.real(np.trace(np.kron(XY, X) @ omega_state))

    else:
        tr_x_omega = np.zeros(num_alpha)
        tr_y_omega = np.zeros(num_alpha)
        for i in range(num_alpha):
            tr_x_omega[i] = get_exp_helper(expectation_unitary_dict, Xpaulis[i], omega, qcbm_shots, shots)
            tr_y_omega[i] = get_exp_helper(expectation_unitary_dict, Ypaulis[i], omega, qcbm_shots, shots)
        return np.real(np.dot(alpha, tr_x_omega - 1j * tr_y_omega) / 2)

In [ ]:
# precompute terms
rho_squared = get_purity(rho, None)
sigma_squared = get_purity(sigma, None)

In [ ]:
N = 2 ** sys_size
def primal_cost_function_full(params):
    """
    params: in the form of [real alphas, complex alphas, omega_params, lambda]
    """
    # get new parameters
    lmbda = params[-1]
    
    real_alpha = params[:num_alpha]
    complex_alpha = np.multiply(params[num_alpha:2*num_alpha], 1j)
    alpha = np.add(real_alpha, complex_alpha)
    
    omega_params = params[2*num_alpha:]
    omega.set_params(omega_params)
    
    termA = N * real_alpha[0]
    
    # calculate objctive function
    omega_squared = get_purity(omega, qcbm_shots)
    trXX = N * (np.linalg.norm(alpha) ** 2)
    termB = rho_squared + sigma_squared + lmbda ** 2 * omega_squared + 2 * trXX
    
    tr_rho_omega = get_trace_omega(True, qcbm_shots, num_shots)
    tr_sigma_omega = get_trace_omega(False, qcbm_shots, num_shots)
    termC = -2 * lmbda * (tr_rho_omega + tr_sigma_omega + 2 * get_pauli_omega_trace(alpha, qcbm_shots, num_shots))
    
    obj = termA
    penal = termB + termC
    cost = obj - c * penal
    return cost, obj, penal

def primal_cost_function(params):
    cost, _, _ = primal_cost_function_full(params)
    return -cost

In [ ]:
print_every = 100
def primal_callback(x, iteration):
    '''
    Callback function called automatically during optimization. 
    Appends loss value, penalties, and objective value.
    Print out the training status throughout the training.
    '''
    
    global lr
    global lr_wait
    
    global primal_loss
    global primal_penals
    global primal_ojectives
    global primal_iterations
    
    fx, obj, penal = primal_cost_function_full(x)
    primal_loss.append(fx)
    primal_penals.append(penal)
    primal_objectives.append(obj)
    primal_iterations.append(iteration)
        
    if (iteration % print_every == 0):
        lr_wait += 1
        if lr_wait >= 5:
            m, _ = np.polyfit(primal_iterations[-500:], primal_loss[-500:], 1)
            if m < 0:
                lr = max(lr/2,lr_min)
                lr_wait = 0
            else:
                lr = min(lr*1.1,lr_max)
        print(f"Iteration: {iteration}\tCost: {round(fx,4)}\tObj: {round(obj, 4)}\tPenalty: {round(penal, 4)}\tc: {round(c, 4)}\tlr: {round(lr, 4)}")


In [ ]:
# General settings
num_shots = None
qcbm_shots = None

max_iter = 100 * 1000
perturbation = 1e-4
c = 50

lr_max = 1e-2
lr_min = 5e-4

primal_num_layers = 8
primal_qcbm_layers = 3

In [ ]:
num_train = 3
record = 0

iterations_primal_outer = []
costs_primal_outer = []
objs_primal_outer = []
penalties_primal_outer = []

for i in range(num_train):
    lr = lr_max
    lr_wait = 0
    
    lmbda = 1
#     alpha_init = np.ones(num_alpha) / num_alpha
#     alpha_init = np.random.normal(size=num_alpha)
    alpha_init = np.zeros(num_alpha)
    omega = create_cca("general", omega_size, ref_size, primal_num_layers, primal_qcbm_layers)

    # training loop
    print("Expected value:", expected)
    init_params = [*alpha_init, *alpha_init, *omega.get_params(), lmbda]
    init_cost, init_obj, init_penal = primal_cost_function_full(init_params)
    primal_loss = [init_cost]
    primal_penals = [init_penal]
    primal_objectives = [init_obj]
    primal_iterations = [0]
    
    print(f"Iteration {i}, Initial cost : {init_cost}")
    result = gradient_descent(primal_cost_function, init_params, get_gradient, max_iter, callback=primal_callback)
    
    costs_primal_outer.append(primal_loss)
    objs_primal_outer.append(primal_objectives)
    penalties_primal_outer.append(primal_penals)
    iterations_primal_outer.append(primal_iterations)
    
    
    if abs(primal_loss[-1] - expected) <= 0.05:
        dir_name = f"Fidelity_CCA_primal_data/seed{problem_seed}"
        problem_name = "Fid"
        filename = f"Init_{i}"
        save_data_to_file(primal_loss, primal_penals, primal_objectives, problem_name, dir_name, filename)
        record += 1
        
print("Number of successful trails: ", record)

# Plots

In [ ]:
# plt.rc('font', family='Times New Roman')
plt.rc('font', family='Arial')
plt.rc('axes', titlesize=24)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('legend', framealpha = 1)
plt.rc('legend', shadow = True)

In [ ]:
# Size of the smoothing window
window_size_primal = 500
window_size_dual = 500

iterations_primal = np.arange(window_size_primal, len(iterations_primal_outer[0])+1)
iterations_dual = np.arange(window_size_dual, len(iterations_dual_outer[0])+1)

costs_primal_smooth = get_smooth(costs_primal_outer[:-1], window_size_primal)
costs_dual_smooth = get_smooth(costs_dual_outer[:-1], window_size_dual)

In [ ]:
# IQR plot
max_iter_num = max(len(iterations_primal_outer[0]), len(iterations_dual_outer[0]))

costs_primal_median = np.median(costs_primal_smooth, axis = 0)
costs_primal_max = np.percentile(costs_primal_smooth, 75, axis = 0)
costs_primal_min = np.percentile(costs_primal_smooth, 25, axis = 0)

costs_dual_median = np.median(costs_dual_smooth, axis = 0)
costs_dual_max = np.percentile(costs_dual_smooth, 75, axis = 0)
costs_dual_min = np.percentile(costs_dual_smooth, 25, axis = 0)

# Plotting the line graph
expected_line = np.zeros(max_iter_num) + expected
plt.plot(iterations_primal, costs_primal_median, color = 'darkblue', linewidth = 1.5)
plt.plot(iterations_dual, costs_dual_median, color = 'purple', linewidth = 1.5)
plt.plot(np.arange(max_iter_num), expected_line, linestyle='dashed', color = 'deeppink', markersize = 1, linewidth = 2)

plt.legend(['QSlack Primal','QSlack Dual', 'True Value'])

plt.fill_between(iterations_primal, costs_primal_max, costs_primal_min, color = 'darkblue', alpha = .2)
plt.fill_between(iterations_dual, costs_dual_max, costs_dual_min, color = 'plum', alpha = .2)

plt.xlabel('Iteration')
plt.ylabel('Objective Function Value')
plt.grid(True)
# plt.ylim(-8, 3)
plt.xlim(0, max_iter_num)

# Display the graph
plt.show()